In [ ]:
# Libraries
import pandas as pd
import seaborn as sns
import openpyxl

In [ ]:
# Loading the file
cdr = pd.read_excel('data/Sample CDR.xlsx')

In [ ]:
# CDR
print(f'CDR shape: {cdr.shape}')
print(f'\nCDR columns: \n{cdr.columns}')
print(f'\nCDR info: \n{cdr.info()}')
print(f'\nCDR describe: \n{cdr.describe()}')